In [1]:
%load_ext autotime

import os, math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')
from tqdm import tqdm, trange

In [2]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Input, Conv1D, LSTM, MaxPooling1D
from sklearn.metrics import mean_squared_error, mean_absolute_error

def model(test_data, train_data):
    test_time = test_data['time']
    test_gl_value = test_data['gl_value']
    
    test_data.drop(columns = ['time'], inplace = True)
    train_data.rename(columns = {'bolus_type_normal dual':'bolus_type_normal_dual', 'bolus_type_square dual':'bolus_type_square_dual'}, inplace = True)
    train_data.drop(columns = ['time'], inplace = True)
    
    empty_train_col = [0]*len(train_data)
    for i, item in enumerate(test_data.columns):
        if item not in train_data.columns:
            train_data.insert(i, item, empty_train_col)

    empty_test_col = [0]*len(test_data)
    for i, item in enumerate(train_data.columns):
        if item not in test_data.columns:
            test_data.insert(i, item, empty_test_col)
    
    X_data = train_data.drop(columns = ['gl_predict'])
    y_data = train_data[['gl_predict']]
    input_dim = X_data.shape[1]
    
    scaler_x = MinMaxScaler()
    scaler_y = MinMaxScaler()
    scaler_x.fit(X_data)
    scaler_y.fit(y_data)
    scaled_X_train_data = scaler_x.transform(X_data)
    scaled_y_train_data = scaler_y.transform(y_data)
    scaled_X_train_data = np.reshape(scaled_X_train_data, (scaled_X_train_data.shape[0], 1, scaled_X_train_data.shape[1]))
    
    model = Sequential()
    model.add(Conv1D(filters = 32, kernel_size = 1, activation = 'relu', input_shape = (scaled_X_train_data.shape[1], scaled_X_train_data.shape[2])))
    model.add(LSTM(128, return_sequences = False))
    model.add(Dense(150, activation = 'relu'))
    model.add(Dropout(0.2))
    model.add(Dense(100, activation = 'relu'))
    model.add(Dropout(0.15))
    model.add(Dense(50, activation = 'relu'))
    model.add(Dense(16, activation = 'relu'))
    model.add(Dense(1, activation = 'relu'))
    model.compile(loss = 'mse', optimizer = 'adam')

    model.fit(scaled_X_train_data, scaled_y_train_data, epochs = 50, batch_size = 32, shuffle = False)
    
    X_test_data = test_data.drop(columns = ['gl_predict'])
    y_test_data = test_data[['gl_predict']]
    scaled_X_test_data = scaler_x.transform(X_test_data)
    scaled_X_test_data = np.reshape(scaled_X_test_data, (scaled_X_test_data.shape[0], 1, scaled_X_test_data.shape[1]))
    prediction = model.predict(scaled_X_test_data, batch_size = 32)
    scaled_prediction = scaler_y.inverse_transform(prediction)
    
    final_output = pd.DataFrame(columns = ['timestamp', 'predicted_BGL_value'])
    final_output['timestamp'] = test_time
#     final_output['true_BGL_value'] = test_gl_value
    final_output['predicted_BGL_value'] = scaled_prediction
    
    return (final_output, math.sqrt(mean_squared_error(scaled_prediction, y_test_data)), mean_absolute_error(scaled_prediction, y_test_data))

Using TensorFlow backend.


time: 3.67 s


In [3]:
test_data_list = ['540-ws-training.csv', '544-ws-training.csv', '552-ws-training.csv', '567-ws-training.csv', '584-ws-training.csv', '596-ws-training.csv']

rmse_list = {}
mae_list = {}
for file in os.listdir('../data/derived/60_min_complete_dataset/'):
    if file in test_data_list:
        train_data = pd.read_csv('../data/derived/60_min_complete_dataset/'+file)
        test_data = pd.read_csv('../data/derived_test/60_min_complete_dataset/'+file[0:7]+'testing.csv')
        final_output, rmse_error, mae_error = model(test_data, train_data)
        rmse_list[file] = rmse_error
        mae_list[file] = mae_error
        print ('{} - {}'.format(file, rmse_error))
        print ('{} - {}'.format(file, mae_error))
        final_output.to_csv('../data/output/conv_lstm/'+file[0:7]+'result_60min.csv')

Epoch 1/50
1103/1103 [==============================] - 2s 2ms/step - loss: 0.0661
Epoch 2/50
1103/1103 [==============================] - 0s 267us/step - loss: 0.0300
Epoch 3/50
1103/1103 [==============================] - 0s 222us/step - loss: 0.0201
Epoch 4/50
1103/1103 [==============================] - 0s 320us/step - loss: 0.0157
Epoch 5/50
1103/1103 [==============================] - 0s 251us/step - loss: 0.0150
Epoch 6/50
1103/1103 [==============================] - 0s 283us/step - loss: 0.0138
Epoch 7/50
1103/1103 [==============================] - 0s 285us/step - loss: 0.0132
Epoch 8/50
1103/1103 [==============================] - 0s 284us/step - loss: 0.0125
Epoch 9/50
1103/1103 [==============================] - 0s 222us/step - loss: 0.0136
Epoch 10/50
1103/1103 [==============================] - 0s 258us/step - loss: 0.0127
Epoch 11/50
1103/1103 [==============================] - 0s 240us/step - loss: 0.0125
Epoch 12/50
1103/1103 [==============================] - 0s 332us

1127/1127 [==============================] - 0s 204us/step - loss: 0.0094
Epoch 46/50
1127/1127 [==============================] - 0s 271us/step - loss: 0.0096
Epoch 47/50
1127/1127 [==============================] - 0s 296us/step - loss: 0.0094
Epoch 48/50
1127/1127 [==============================] - 0s 300us/step - loss: 0.0094
Epoch 49/50
1127/1127 [==============================] - 0s 299us/step - loss: 0.0097
Epoch 50/50
1127/1127 [==============================] - 0s 215us/step - loss: 0.0094
567-ws-training.csv - 37.03151524801891
567-ws-training.csv - 27.577353236674213
Epoch 1/50
1135/1135 [==============================] - 3s 2ms/step - loss: 0.0640
Epoch 2/50
1135/1135 [==============================] - 0s 272us/step - loss: 0.0298
Epoch 3/50
1135/1135 [==============================] - 0s 341us/step - loss: 0.0252
Epoch 4/50
1135/1135 [==============================] - 0s 267us/step - loss: 0.0176
Epoch 5/50
1135/1135 [==============================] - 0s 238us/step - loss:

924/924 [==============================] - 0s 293us/step - loss: 0.0135
Epoch 40/50
924/924 [==============================] - 0s 288us/step - loss: 0.0168
Epoch 41/50
924/924 [==============================] - 0s 330us/step - loss: 0.0200
Epoch 42/50
924/924 [==============================] - 0s 352us/step - loss: 0.0225
Epoch 43/50
924/924 [==============================] - 0s 303us/step - loss: 0.0232
Epoch 44/50
924/924 [==============================] - 0s 265us/step - loss: 0.0170
Epoch 45/50
924/924 [==============================] - 0s 275us/step - loss: 0.0140 0s - loss: 0.0
Epoch 46/50
924/924 [==============================] - 0s 314us/step - loss: 0.0126
Epoch 47/50
924/924 [==============================] - 0s 423us/step - loss: 0.0120
Epoch 48/50
924/924 [==============================] - 0s 285us/step - loss: 0.0120
Epoch 49/50
924/924 [==============================] - 0s 338us/step - loss: 0.0128
Epoch 50/50
924/924 [==============================] - 0s 360us/step - lo

1092/1092 [==============================] - 0s 259us/step - loss: 0.0130
Epoch 33/50
1092/1092 [==============================] - 0s 310us/step - loss: 0.0123
Epoch 34/50
1092/1092 [==============================] - 0s 251us/step - loss: 0.0125
Epoch 35/50
1092/1092 [==============================] - 0s 283us/step - loss: 0.0124
Epoch 36/50
1092/1092 [==============================] - 0s 265us/step - loss: 0.0120
Epoch 37/50
1092/1092 [==============================] - 0s 303us/step - loss: 0.0119
Epoch 38/50
1092/1092 [==============================] - 0s 334us/step - loss: 0.0121
Epoch 39/50
1092/1092 [==============================] - 0s 381us/step - loss: 0.0119
Epoch 40/50
1092/1092 [==============================] - 0s 301us/step - loss: 0.0122
Epoch 41/50
1092/1092 [==============================] - 0s 378us/step - loss: 0.0123
Epoch 42/50
1092/1092 [==============================] - 0s 376us/step - loss: 0.0118
Epoch 43/50
1092/1092 [==============================] - 0s 341us/

In [4]:
rmse_list

{'584-ws-training.csv': 40.169175435312475,
 '567-ws-training.csv': 37.03151524801891,
 '596-ws-training.csv': 26.92855464848946,
 '552-ws-training.csv': 27.542260232723127,
 '544-ws-training.csv': 105.46076550491148,
 '540-ws-training.csv': 39.61574759405369}

time: 10.5 ms


In [5]:
mae_list

{'584-ws-training.csv': 30.565070596455293,
 '567-ws-training.csv': 27.577353236674213,
 '596-ws-training.csv': 20.07910155668712,
 '552-ws-training.csv': 20.37609164530611,
 '544-ws-training.csv': 29.741567190629347,
 '540-ws-training.csv': 30.20977360929748}

time: 4.58 ms


In [6]:
from statistics import pstdev, mean

rmse_values = rmse_list.values()
mae_values = mae_list.values()

print ('Mean - RMSE : {}'.format(mean(rmse_values)))
print ('Standard Deviation - RMSE : {}'.format(pstdev(rmse_values)))

print ('Mean - MAE : {}'.format(mean(mae_values)))
print ('Standard Deviation - MAE : {}'.format(pstdev(mae_values)))

Mean - RMSE : 46.124669777251526
Standard Deviation - RMSE : 27.065009627609562
Mean - MAE : 26.424826305841595
Standard Deviation - MAE : 4.484267886820347
time: 13.5 ms
